In [ ]:
import torch
from timm import create_model
from torchvision import transforms
from PIL import Image
import os

# Load ViT pretrained on ImageNet
model = create_model('vit_base_patch16_224', pretrained=True)
model.eval()


#image = Image.open("../fyp/skinType/patches128x128")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
root_folder = "./skintypepatches 128x128"
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)
    if os.path.isdir(subfolder_path):
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            if os.path.isfile(img_path) and filename.endswith((".jpg", ".png", ".jpeg")):
                try:
                    image = Image.open(img_path).convert("RGB")
                    img_tensor = transform(image).unsqueeze(0)
                    print(f"Processed: {img_path}")
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
# Inference
with torch.no_grad():
    logits = model(img_tensor)

print("Logits shape:", logits.shape)  # [1, 1000] for 1000 ImageNet classes


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches
import os

# Load CSV
df = pd.read_csv('pretrained_overall_result.csv')

# Set correct column names
lr_col = 'Learning Rate'
bs_col = 'Batch Size'
opt_col = 'Optimizer'
epoch_col = 'Epoch'

# Metrics to plot
metrics = [ 
    'Train Loss', 'Test Loss', 'Val Loss',
    'Train Acc', 'Test Acc', 'Val Acc',
    'Train Precision', 'Test Precision', 'Val Precision',
    'Train Recall', 'Test Recall', 'Val Recall',
    'Train F1 Score', 'Test F1 Score', 'Val F1 Score',
    'Train ROC AUC', 'Test ROC AUC', 'Val ROC AUC',
    'Train RMSE', 'Test RMSE', 'Val RMSE'
]

# Create unique configuration name
df['config'] = df[lr_col].astype(str) + '_lr_' + \
               df[bs_col].astype(str) + '_bs_' + \
               df[opt_col]

# Set plot style
sns.set(style='whitegrid')

# Create a directory to save images
image_dir = "metric_plots"
os.makedirs(image_dir, exist_ok=True)

# Create Word document
doc = Document()
doc.add_heading('Training Metrics Report', 0)

# Generate plots and add to document
for metric in metrics:
    plt.figure(figsize=(10, 6))
    for config_name, group in df.groupby('config'):
        plt.plot(group[epoch_col], group[metric], label=config_name)

    plt.title(metric)
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend(title='Config', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    # Save plot as image
    image_path = os.path.join(image_dir, f"{metric.replace(' ', '_')}.png")
    plt.savefig(image_path)
    plt.close()

    # Add title and image to Word doc
    doc.add_heading(metric, level=1)
    doc.add_picture(image_path, width=Inches(6.5))  # Adjust width if needed

# Save the Word document
doc.save('Training_Metrics_Report.docx')
print("Word document 'Training_Metrics_Report.docx' created with all plots.")


Word document 'Training_Metrics_Report.docx' created with all plots.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
import os

# Load the data
df = pd.read_csv("pretrained_overall_result.csv")

# Combine hyperparameters into a config string for reference (not used in graph)
df['Config'] = df.apply(lambda row: f"LR={row['Learning Rate']} | BS={row['Batch Size']} | Opt={row['Optimizer']}", axis=1)

# Set Seaborn style
sns.set_theme(style="whitegrid")

# List of metrics to visualize
metrics = [
    'Train Loss', 'Test Loss', 'Val Loss',
    'Train Acc', 'Test Acc', 'Val Acc',
    'Train Precision', 'Test Precision', 'Val Precision',
    'Train Recall', 'Test Recall', 'Val Recall',
    'Train F1 Score', 'Test F1 Score', 'Val F1 Score',
    'Train ROC AUC', 'Test ROC AUC', 'Val ROC AUC',
    'Train RMSE', 'Test RMSE', 'Val RMSE'
]

# Create a folder for images
img_folder = "metric_images"
os.makedirs(img_folder, exist_ok=True)

# Create Word document
doc = Document()
doc.add_heading("Model Evaluation Metrics Report", 0)

# Loop over each metric, create and save plot, insert into doc
for metric in metrics:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Plot by Learning Rate
    sns.lineplot(data=df, x='Epoch', y=metric, hue='Learning Rate', marker='o', ax=axes[0])
    axes[0].set_title(f"{metric} vs Epoch (Learning Rate)")
    axes[0].legend(title="LR", fontsize=7)

    # Plot by Batch Size
    sns.lineplot(data=df, x='Epoch', y=metric, hue='Batch Size', marker='o', ax=axes[1])
    axes[1].set_title(f"{metric} vs Epoch (Batch Size)")
    axes[1].legend(title="BS", fontsize=7)

    # Plot by Optimizer
    sns.lineplot(data=df, x='Epoch', y=metric, hue='Optimizer', marker='o', ax=axes[2])
    axes[2].set_title(f"{metric} vs Epoch (Optimizer)")
    axes[2].legend(title="Opt", fontsize=7)

    # Save figure
    image_path = os.path.join(img_folder, f"{metric.replace(' ', '_')}.png")
    plt.tight_layout()
    fig.savefig(image_path, dpi=300)
    plt.close(fig)

    # Add to Word doc
    doc.add_heading(metric, level=1)
    doc.add_picture(image_path, width=Inches(6.5))  # Use full width, fits well

# Save final document
doc.save("Metric_Evaluation_Report.docx")
print("✅ All metric graphs saved and Word document generated as 'Metric_Evaluation_Report.docx'.")


✅ All metric graphs saved and Word document generated as 'Metric_Evaluation_Report.docx'.
